## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


In [1]:
#importing all relevant packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline


In [2]:
#Step 1: Compute camera calibration matrix and distortion coefficients based on a set of ChessBoard images

def calibrate_camera():
    #Function that outputs calicbration matrix and distortion coefficients from saved chess board images  
    
    # Initiate object points for correting for image distortion 
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return (mtx,dist)

def undistort_image(img, camera_calib):
    #Function that uses the calibration matrix and distortion coefficients to undistort image 
    dst = cv2.undistort(img, camera_calib[0], camera_calib[1], None, camera_calib[0])
    return dst


In [3]:
#Calibrate camera, store calibration matrix and distortion coefficients in global variable camera calib
camera_calib = calibrate_camera()


In [4]:
#Step 2:define region of interest of the image:

def region_of_interest(img,vertices):
    #Function that crops out region of interest 
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [5]:
#Step3: Define thresholding functions based on gradient and color

def color_gradient_mag(hsv_img,kernel=3, mag_thresh = (0,255)):
    #Function that thresholds the image based on gradient in HSV color space 
    
    #Extract color channels from input HSV image
    h = hsv_img[:,:,0]
    s = hsv_img[:,:,1]
    v = hsv_img[:,:,2]
    
    #Calculate the gradient in x&y for each color channel
    h_sobelx = cv2.Sobel(h,cv2.CV_64F,1,0,ksize=kernel)
    h_sobely = cv2.Sobel(h,cv2.CV_64F,0,1,ksize=kernel)
    
    s_sobelx = cv2.Sobel(s,cv2.CV_64F,1,0,ksize=kernel)
    s_sobely = cv2.Sobel(s,cv2.CV_64F,0,1,ksize=kernel)
    
    v_sobelx = cv2.Sobel(v,cv2.CV_64F,1,0,ksize=kernel)
    v_sobely = cv2.Sobel(v,cv2.CV_64F,0,1,ksize=kernel)
    
    #calculate the differentiol for each color channel
    dh_s = (h_sobelx+h_sobely) *np.pi/255*s #convert to radians, and muliply by s since s is radius in HSV, multiplying radius
                                            #by angle gives the segment length
    ds = s_sobelx+s_sobely
    dv = v_sobelx+v_sobely
    
    #calculate color gradient magnitude based on the values for each differentials
    color_mag = np.sqrt(dh_s**2+ds**2+dv**2)
    #scale magnitude
    scaled_mag = np.uint8(255*color_mag/np.max(color_mag))
    #threshold image 
    binary = np.zeros_like(scaled_mag)
    binary[(scaled_mag>=mag_thresh[0])&(scaled_mag<=mag_thresh[1])]=1

    return binary

def color_gradient_dir(hsv_img,kernel=3, dir_thresh = (0,255)):
    #Function that threholds the image based on gradient direction in HSV color space
    
    #Extract color channels from input HSV image
    h = hsv_img[:,:,0]
    s = hsv_img[:,:,1]
    v = hsv_img[:,:,2]
    
    #Calculate the gradient in x&y for each color channel
    h_sobelx = cv2.Sobel(h,cv2.CV_64F,1,0,ksize=kernel)
    h_sobely = cv2.Sobel(h,cv2.CV_64F,0,1,ksize=kernel)
    
    s_sobelx = cv2.Sobel(s,cv2.CV_64F,1,0,ksize=kernel)
    s_sobely = cv2.Sobel(s,cv2.CV_64F,0,1,ksize=kernel)
    
    v_sobelx = cv2.Sobel(v,cv2.CV_64F,1,0,ksize=kernel)
    v_sobely = cv2.Sobel(v,cv2.CV_64F,0,1,ksize=kernel)
    
    #calculate differentials dx and dy based on color gradients 
    dx = 1/h_sobelx/s+1/s_sobelx+1/v_sobelx #multipy H_sobel x by s since h is an angular measurement and s is radius to get segment length 
    dy = 1/h_sobely/s+1/s_sobely+1/v_sobely
    
    #calculate absolute values for each of the gradient differentials 
    abs_x= np.absolute(dx)
    abs_y= np.absolute(dy)
    #calculate gradient direction
    grad_dir = np.arctan2(abs_y, abs_x)
    #threshold image
    binary = np.zeros_like(grad_dir)
    binary[(grad_dir>=dir_thresh[0])&(grad_dir<=dir_thresh[1])]=1
    
    return binary

def color_space_select(hsv_img):
    #Function that Threshold's the image based on location in color space 
    
    #Extract color channels
    h = hsv_img[:,:,0]
    s = hsv_img[:,:,1]
    v = hsv_img[:,:,2]
    #threshold image
    binary = np.zeros_like(h)
    binary[(v+0.75*s-255)>=0]=1
    return binary
    

In [6]:
#Step 4: Apply perspective transform to get a birds eye view of the road
def perspective_transform(img,verticies):
    #Function that transforms image into a bird's eye view of the road 
    
    #define source and destination vertices
    src = np.float32(verticies) 
    dst = np.float32([[0,720],[0,0],[1280,0],[1280,720]])
    #Calculate transformation matrix
    M = cv2.getPerspectiveTransform(src, dst)
    #Transform image 
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped 


In [26]:
#Step 5: Create class for storing detected lane lines 
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        #Most recent fit coefficients
        self.recent_fit = 0
        #X-intercept of lane line
        self.x_base = 0
        #lane_curvature
        self.curvature = 0
        #Number of previous good fits detected (max 10)
        self.previous_fits = []
        #Avg coefficients of previous good fits detected
        self.fit_average = 0 
        #Number of previous no good fits detected 
        self.ngood_no = 0
    
    def expected_lane_region(self,binary_warped):
        #Function that crops out the region that the lane lines are expected to appear based on average polynomial
        
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        lane_inds = ((nonzerox > (self.fit_average[0]*(nonzeroy**2) + self.fit_average[1]*nonzeroy + 
                        self.fit_average[2] - margin)) & (nonzerox < (self.fit_average[0]*(nonzeroy**2) + 
                        self.fit_average[1]*nonzeroy +self.fit_average[2] + margin)))
    
        lane_x = nonzerox[lane_inds]
        lane_y = nonzeroy[lane_inds] 
        
        
        result = np.zeros_like(binary_warped)
        
        result[lane_y,lane_x]=1

        return result
    
    def fit_polynomial(self, binary_warped,nwindows,margin,minpix):
        #Function that  fits polynomial to transformed images 

        lane_x_base = self.x_base


        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated later for each window in nwindows
        lane_x_current = lane_x_base

        # Create empty lists to receive left and right lane pixel indices
        lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            # Find the four below boundaries of the window ###
            win_x_low = lane_x_current - margin  # Update this
            win_x_high = lane_x_current + margin  # Update this
    
            # Identify the nonzero pixels in x and y within the window ###
            good_lane_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]
           
            # Append these indices to the list
            lane_inds.append(good_left_inds)

            #If you found > minpix pixels, recenter next window ###
            if len(good_lane_inds) > minpix:
                lane_x_current = np.int(np.mean(nonzerox[good_lane_inds]))
            
        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            lane_inds = np.concatenate(lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        lane_x = nonzerox[lane_inds]
        lane_y = nonzeroy[lane_inds] 
        
        lane_fit = np.polyfit(lane_y,lane_x,2)
        
        #update self.recent_fit, and self.x_base:
        self.recent_fit = lane_fit
        self.x_base = lane_fit[2]
        
        return lane_fit
    def line_curvature(self):
        self.curvature = (1+(2*self.recent_fit[0]+self.recent_fit[1])**2)**(3/2)/abs(2*self.recent_fit[0])
        return self.curvature
    
    def check_sanity_and_update(self,x_base_thesh,curvature_thesh):
        if (self.curvature >curvature_thresh[0])and (self.curvature < curvature_thresh[1]) and (self.x_base >x_base_thresh[0]) and (self.x_base < x_base_thresh[1]):
            self.detected = True
            self.previous_fits.append(self.recent_fit)
            if len(self.previous_fits)>10:
                self.previous_fits.remove(0)
            self.fit_avg = sum(self.previous_fits)/len(self.previous_fits)
            self.ngood_no = 0
        else:
            self.detected = False
            if self.ngood_no >5:
                self.fit_avg = []
            else:
                self.ngood+=1
            

In [7]:
#Step 6: Fit polynomial to detected lane lines

 

def expected_lane_region(binary_warped,left_fit,right_fit, margin):
    #Function that crops out the region that the lane lines are expected to appear
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    result = np.zeros_like(binary_warped)
    
    result[lefty,leftx]=1
    result[righty,rightx]=1
    
    return result

def detect_lanes(binary_warped,left_lane,right_lane):
    
def plot_detected_lane(binary_warped, left_fit, right_fit):
    
    result = np.ones_like(binary_warped)
    
    zero = result.nonzero()
    zeroy = np.array(zero[0])
    zerox = np.array(zero[1])
    
    lane_inds = ((zerox > (left_fit[0]*(zeroy**2) + left_fit[1]*zeroy + 
                    left_fit[2])) & (zerox < (right_fit[0]*(zeroy**2) + 
                    right_fit[1]*zeroy + right_fit[2])))
    lanex = zerox[lane_inds]
    laney = zeroy[lane_inds] 
    
    result = np.zeros_like(binary_warped)
    result[laney,lanex]=1
    
    return result
    

In [8]:
#funtcion f or transforming the detected region back to the perspective view of the road
def transform_back(img):
    dst = np.float32([[110,720],[550,460],[750,460],[1270,720]])
    src = np.float32([[0,720],[0,0],[1280,0],[1280,720]])
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped 

In [22]:
counter = 0
prior_poly = []

def pipeline(img, camera_calib):
    #Function containing hte pipline for processing image
    
    #Declare global variables
    global counter
    global prior_poly
    
    #Undistort image (Correct for lens distortion)
    undist = undistort_image(img, camera_calib)
    
    #Crop out region of interest
    vertices = np.array([[[100,700],[550,450],[750,450],[1280,700]]])
    ROI = region_of_interest(undist,vertices)
    
    #Convert image to HSV
    hsv_img = cv2.cvtColor(ROI,cv2.COLOR_RGB2HSV)
    
    #Threshold image based on gradient magnitude and direction as well as location in color space 
    grad_binary = color_gradient_mag(hsv_img,3, (15,255))
    dir_binary = color_gradient_dir(hsv_img,11, (0/180*np.pi,60/180*np.pi))
    color_binary = color_space_select(hsv_img)
    comb_binary = np.zeros_like(color_binary)
    comb_binary[(color_binary==1)|((grad_binary==1)&(dir_binary==1))]=1
    
    #Transform image to get a bird's eye view of the road
    binary_warped = perspective_transform(comb_binary,[[110,700],[550,460],[750,460],[1270,700]])
    #For testing only: Output image -> binary warped
    out_img = np.dstack((binary_warped*255,binary_warped*255,binary_warped*255))
    
    """
    if counter==0:
        expected_img = binary_warped
        left_fit , right_fit = fit_polynomial(binary_warped)
        prior_poly =  [left_fit , right_fit]
        counter +=1
    else:
        expected_img = expected_lane(binary_warped,prior_poly[0],prior_poly[1])
        left_fit , right_fit = fit_polynomial(binary_warped)
        prior_poly =  [left_fit , right_fit]
    
    lane = plot_detected_lane(binary_warped, left_fit, right_fit)
    lane_unwarped = transform_back(lane)
    lane_ingreen = np.dstack((np.zeros_like(lane_unwarped),lane_unwarped*255,np.zeros_like(lane_unwarped)))
    out_img = cv2.addWeighted(undist, 1, lane_ingreen, 0.3, 0)
    """
    
    return out_img


In [23]:

def process_image(img):
    #Function that processes image using defined pipeline
    result = pipeline(img, camera_calib)
    return result

In [24]:
white_output = 'output_images/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
#clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

"""
img = mpimg.imread('test_images/test6.jpg')
plt.imshow(img)
print(img.shape)
plt.show()

f_img = color_gradient(img,11,(50,255),(0,60/180*np.pi))
plt.imshow(f_img)
plt.show()"""


[MoviePy] >>>> Building video output_images/project_video_output.mp4
[MoviePy] Writing video output_images/project_video_output.mp4



 80%|████████████████████████████████        | 101/126 [01:15<00:18,  1.36it/s]


 99%|███████████████████████████████████████▋| 125/126 [01:33<00:00,  1.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video_output.mp4 

Wall time: 1min 34s


"\nimg = mpimg.imread('test_images/test6.jpg')\nplt.imshow(img)\nprint(img.shape)\nplt.show()\n\nf_img = color_gradient(img,11,(50,255),(0,60/180*np.pi))\nplt.imshow(f_img)\nplt.show()"

In [25]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
#Archived functions

"""
def abs_sobel_thresh(gray_img, orient='x', thresh_min=0, thresh_max=255):
    # Apply the following steps to img
    
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        x = 1
        y=0
    elif orient == 'y':
        x = 0
        y = 1
    else:
        raise ValueError 
        
    sobelx = cv2.Sobel(gray_img,cv2.CV_64F,x,y)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobelx = np.absolute(sobelx)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sxbinary = np.zeros_like(scaled_sobel)
    # 6) Return this mask as your binary_output image
    sxbinary [(scaled_sobel >= thresh_min) & (scaled_sobel <=thresh_max)] = 1
    
    return sxbinary

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
  
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    grad = np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*grad/np.max(grad))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    binary = np.zeros_like(scaled_sobel)
    # 5) Create a binary mask where mag thresholds are met
    binary[(scaled_sobel>=mag_thresh[0])&(scaled_sobel<=mag_thresh[1])]=1
    # 6) Return this mask as your binary_output image
    return binary

def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx= np.absolute(sobelx)
    abs_sobely= np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    #scaled_sobel = np.uint8(255*grad_dir/np.max(grad_dir))
    # 5) Create a binary mask where direction thresholds are met
    binary = np.zeros_like(grad_dir)
    binary[(grad_dir>=thresh[0]) & (grad_dir<=thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary

def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls_img= cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    s = hls_img[:,:,2]
    s_masked_img = np.zeros_like(s)
    s_masked_img[(s>thresh[0])&(s<=thresh[1])] =1
    # 3) Return a binary image of threshold result
    return s_masked_img
    
    """